# Abres de décisions

Un extrait du livre d'Aurélien Géron

## Intro

Les arbres de décision sont des algorithmes d'apprentissage automatique polyvalents qui peuvent effectuer des tâches de classification et de régression, et même des tâches à sorties multiples. Ce sont des algorithmes très puissants, capables de s'adapter à des ensembles de données complexes.  Les arbres de décision sont également les composants fondamentaux des forêts aléatoires, qui sont parmi les algorithmes d'apprentissage automatique les plus puissants disponibles aujourd'hui. 

* Nous commencerons par discuter de la façon de les entraîner, de les visualiser et de faire des prédictions.
* Ensuite, nous passerons en revue l'algorithme d'entraînement CART utilisé par Scikit-Learn
* nous discuterons de la manière de régulariser les arbres
* Nous les utiliserons aussi pour des tâches de régression. 
* Enfin, nous discuterons de certaines des limites des arbres de décision 


### Installation de graphviz

graphviz est une bibliothèqur pour visualiser les graphes. 

In [ ]:
"on installe la bibliothèque"
!apt-get install graphviz

In [ ]:
"on installe le wrapper python"
!pip install graphviz

Remarque: quand j'ai voulu tester d'installer `graphiz` en local (donc pas avec colab), j'ai du changer des droits d'accès:
      
      sudo chmod 777 /usr/local/include
   
(c'était réclamé dans le message d'erreur)

### The top-cell

In [ ]:
%reset -f


In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import numpy as np
import pandas as pd
import graphviz

import sklearn.linear_model
import sklearn.datasets
import sklearn.tree

## Classification

### Entrainement et visualisation d'un arbre de décision

Pour comprendre les arbres de décision, il suffit d'en construire un et de voir comment il fait des prédictions. Le code suivant entraîne un `DecisionTreeClassifier` sur le fameux jeu de données de l'iris.

In [ ]:
iris = sklearn.datasets.load_iris()
X_iris = iris.data[:, 2:] # petal length and width
y_iris = iris.target

tree_clf = sklearn.tree.DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_iris, y_iris)

In [ ]:
dot_data =sklearn.tree.export_graphviz(
        tree_clf,
        out_file=None, #vous pouvez aussi mettre un chemin d'accès pour que les données du graph soient sauvegardées
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

graph = graphviz.Source(dot_data) 
graph

***A vous:*** Observez l'arbre ci-dessus. Supposez que vous avez un iris tel que

* petal length = 3
* petal width = 1

Dans quel espèce faut-il le classer? 

***Note:*** L'une des nombreuses qualités des arbres décisionnels est qu'ils nécessitent très peu de préparation de données. En particulier, ils ne nécessitent pas du tout de mise à l'échelle ou de centrage des variables.

Voici la signification des étiquettes sur l'arbre :

* `samples` : compte le nombre d'instances du jeu `train`. Par exemple, 100 instances ont une longueur de pétale supérieure à 2,45 cm (profondeur 1, droite), dont 54 ont une largeur de pétale inférieure à 1,75 cm (profondeur 2, gauche). 
* `value` : donne la répartition des classes : par exemple, le nœud en bas à droite contient 0 Iris-Setosa, 1 Iris-Versicolor, et 45 Iris-Virginica.
* `gini` : est un indice donnant mesurant l'impureté. Il vaut notamment 0 quand un noeud ne contient que des instance d'une même classe. Et de manière général,  l'indice de Gini est donné par :
$$
G = 1- \sum_k p^2_{k}
$$
où $p_{k}$ est la proportion des instances de la classe $k$ dans le noeud considéré.



***A vous:*** Vérifiez $(2\heartsuit)$ le calcul des différents indices de Gini. Utilisez python comme une calculatrice. 



### Frontière de décision

In [ ]:
def plot_decision_boundary(ax,clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    ax.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        ax.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        ax.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris-Setosa")
        ax.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris-Versicolor")
        ax.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris-Virginica")
        ax.axis(axes)
    if iris:
        ax.set_xlabel("Petal length", fontsize=14)
        ax.set_ylabel("Petal width", fontsize=14)
    else:
        ax.set_xlabel(r"$x_1$", fontsize=18)
        ax.set_ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        ax.legend(loc="lower right", fontsize=14)


In [ ]:
fig,ax=plt.subplots(figsize=(12,6))
plot_decision_boundary(ax,tree_clf,X_iris,y_iris)

#ajoutons manuellement les frontières
ax.plot([2.45, 2.45], [0, 3], "k-", linewidth=2,label="depth=0")
ax.plot([2.45, 7.5], [1.75, 1.75], "k--", linewidth=2,label="depth=1")
ax.plot([4.95, 4.95], [0, 1.75], "k:", linewidth=2,label="depth=2")
ax.plot([4.85, 4.85], [1.75, 3], "k:")
ax.legend();

### Interprétation du modèle : boîte blanche contre boîte noire

Comme vous pouvez le constater, les arbres de décision sont assez intuitifs et leurs décisions sont faciles à interpréter. Ces modèles sont souvent qualifiés de "boîte blanche". En revanche, les Forêts aléatoires ou les réseaux de neurones sont généralement considérés comme des  boîtes noires. Ils font d'excellentes prédictions mais il est difficile d'expliquer en termes simples pourquoi les prédictions ont été faites. Par exemple, si un réseau neuronal dit qu'une personne particulière apparaît sur une image, il est difficile de savoir ce qui a réellement contribué à cette prédiction : le modèle a-t-il reconnu les yeux de cette personne ? Sa bouche ? Son nez ? Ses chaussures ? Ou même le canapé sur lequel elle était assise ? À l'inverse, les arbres de décision fournissent des règles de classification simples et agréables qui peuvent même être appliquées manuellement si nécessaire (par exemple, pour la classification des fleurs).



### Estimation des probabilités de classe

Supposons que vous ayez trouvé dans la nature une iris dont les pétales mesurent 5 cm de long et 1,5 cm de large. Prévoyons son espèce:

In [ ]:
print(tree_clf.predict([[5, 1.5]]))
print(tree_clf.predict_proba([[5, 1.5]]))

***A vous:*** Expliquez $(1\heartsuit)$ comment sont calculées les probabilités. Quel $(1\heartsuit)$ est le gros défaut de ce calcul de probabilité. **Aide:** Observez les "decision boundary". Que se passe-t-il près des frontières?

### L'algorithme CART

Scikit-Learn utilise l'algorithme CART (Classification And Regression Tree) pour entrainer les arbres de décision. L'idée est en fait assez simple: l'algorithme divise d'abord l'ensemble `train` en deux sous-ensembles (=noeuds) en utilisant une seule caractéristique $k$ et un seuil $t_k$ (par exemple, "longueur des pétales ≤ 2,45 cm"). Comment choisit-il $k$ et $t_k$ ? Il recherche la paire $(k, t_k)$ qui produit les sous-ensembles les plus purs (pondérés par leur taille). La fonction de coût que l'algorithme tente de minimiser est donnée par 
$$
J(k,t_k)= \frac {m_{\text{left}} } {m} \,  G_{\text{left}} +  \frac {m_{\text{right}} } {m} \,  G_{\text{right}}
$$
où:

* $m_{\text{left/right}}$ est le nombre d'instance dans le noeud  left/right
* $G_{\text{left/right}}$ est l'indice de Gini du noeud left/right 


Une fois l'ensemble `train` divisé en deux, l'algo redivise les sous-ensembles en utilisant la même logique, puis les sous-sous-ensembles et ainsi de suite. La division cesse quand l'arbre atteint la profondeur maximale (définie par l'hyperparamètre "max_depth"), ou bien s'il ne trouve pas de division qui réduira l'impureté. 

Quelques autres hyperparamètres (décrits dans un instant) contrôlent des conditions d'arrêt supplémentaires : `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, et `max_leaf_nodes`.


Comme vous pouvez le voir, l'algorithme CART est un algorithme "glouton": il recherche "avidement" une division optimale au niveau 0, puis répète le processus au niveau 1, puis à chaque niveau. Il ne vérifie pas si une solution moins bonne au niveau 0, produirait une meilleurs solution au niveau 1.


De tels algorithmes  produisent souvent une solution raisonnablement bonne, mais il n'est pas garanti qu'elle soit optimale. Malheureusement, on trouver l'arbre optimal est un problème NP-Complet: il nécessite du temps O(exp(m)) opération (avec $m$ le nombre d'instance), ce qui rend le problème insoluble même pour des ensembles d'entraînement assez petits. C'est pourquoi nous devons nous contenter d'une solution "raisonnablement bonne". 

### Complexité de calcul

Nous écrivons $m$ le nombre d'instance et $n$ le nombre de variable descriptive (`feature`).

Pour faire des prédictions, il faut parcourir l'arbre de décision de la racine à la feuille. Les arbres de décision sont généralement à peu près équilibrés, de sorte que pour traverser l'arbre de décision, il faut passer par  environ $O(log_2(m))$ nœuds. Comme chaque nœud ne nécessite de vérifier que la valeur d'une seule caractéristique, la complexité globale de la prédiction est de seulement $O(log_2(m))$, indépendamment du nombre de variable descriptive. Les prédictions sont donc très rapides, même sur des grands jeux de données.

Cependant, l'algorithme d'apprentissage compare toutes les caractéristiques (ou moins si `max_features` est défini) sur tous les échantillons à chaque nœud. Il en résulte une complexité d'apprentissage de $O(n × m \log_2(m))$. 

### Impureté de Gini ou entropie  ?

Par défaut, pour mesurer l'impureté d'un noeud, l'indice de Gini est utilisé, mais on peut le remplacer par l'entropie, qui est donnée par:
$$
H = - \sum_k p_{k}\log(p_{k})
$$
avec la convention $0\log(0)=0$.

 Alors, faut-il utiliser l'indice de Gini ou l'entropie ? En vérité, la plupart du temps, cela ne fait pas une grande différence: elles mènent à des arbres similaires. L'impureté de Gini est légèrement plus rapide à calculer, c'est donc un bon défaut. Cependant, lorsqu'elles diffèrent, l'impureté de Gini tend à isoler la classe la plus fréquente dans sa propre branche de l'arbre, tandis que l'entropie tend à produire des arbres légèrement plus équilibrés.

### Modèles non paramétriques

Les arbres de décision font très peu d'hypothèses sur les données, contrairement par exemple aux modèles linéaires qui supposent évidemment que les données sont linéairement réparties.  La structure arborescente s'adaptera à toute sorte de données.

 Les modèles comme l'arbre de décision sont souvent appelés modèles non paramétriques, non pas parce qu'il n'a pas de paramètres (la structure de l'arbre de décision est décrite par les paramètres écrits sur le fichier `.dot`) mais parce que le nombre de paramètres n'est pas déterminé avant l'entrainement
 
A l'inverse, un modèle paramétrique tel qu'un modèle linéaire, a un nombre prédéterminé de paramètres, de sorte que son degré de liberté (=flexibilité) est limité, ce qui réduit le risque de sur-ajustement (mais augmente le risque de sous-ajustement).


***A vous:*** Ré-expliquez $(2\diamondsuit)$ avec vos propres mots, la différence entre les modèles paramètrique et non-paramétrique.


### Régularisation 


Pour éviter le sur-apprentissage, vous devez restreindre la complexité de l'arbre. Un arbre moins complexe = un modèle moins flexible = moins de sur-apprentissage. Le fait de limiter la flexibilité d'un modèle s'appelle la "régularisation". 


Voici les différent hyper-paramètres de `DecisionTreeClassifier` qui permettent de régler la complexité de l'arbre produit: 

* `max_depth` : la prolondeur maximale de l'arbre. La valeur par défaut est "None", ce qui signifie illimité. Diminuer ce paramètre est la manière la plus simple de régulariser le modèle. 
* `min_samples_split` : le nombre minimum d'instances qu'un noeud doit avoir pour pouvoir être divisé
* `min_samples_leaf` : le nombre minimum d'instances qu'un noeud terminal (=feuille=leaf) doit avoir
* `min_weight_fraction_leaf` : même chose que `min_samples_leaf` mais exprimé comme une fraction du nombre total d'instances pondérées
* `max_leaf_nodes` : nombre maximum de nœuds terminaux (= feuilles)
* `max_features` : nombre maximum de caractéristiques qui sont évaluées pour le fractionnement à chaque noeud. Par défaut on les prend toute, et sinon elles sont tirés au hasard. 

Augmenter les hyperparamètres `min_xxx` ou réduire les hyperparamètres `max_xxx` régularisera le modèle.



***NOTE:*** D'autres algorithmes fonctionnent en formant d'abord l'arbre de décision sans restrictions, puis en élaguant (supprimant) les nœuds inutiles. Un nœud dont les enfants sont tous des nœuds feuilles est considéré comme inutile si l'amélioration de la pureté qu'il apporte n'est pas statistiquement significative.



***Exemple:*** Regardez ci-dessous : Il est assez évident que le modèle de gauche est surajusté, et le modèle de droite généralisera probablement mieux.

In [ ]:
"le jeu de donné moon est un jeu de donné simulé où les deux classes s'emmèlent"
Xm, ym = sklearn.datasets.make_moons(n_samples=100, noise=0.25, random_state=53)

deep_tree_clf1 = sklearn.tree.DecisionTreeClassifier(random_state=42)
deep_tree_clf2 = sklearn.tree.DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
deep_tree_clf1.fit(Xm, ym)
deep_tree_clf2.fit(Xm, ym)

fix,(ax0,ax1)=plt.subplots(1,2,figsize=(12,6))
plot_decision_boundary(ax0,deep_tree_clf1, Xm, ym, axes=[-1.5, 2.5, -1, 1.5], iris=False)
ax0.set_title("No restrictions", fontsize=16)
plot_decision_boundary(ax1,deep_tree_clf2, Xm, ym, axes=[-1.5, 2.5, -1, 1.5], iris=False)
ax1.set_title("min_samples_leaf = {}".format(deep_tree_clf2.min_samples_leaf), fontsize=14)

## Régression $\hookleftarrow$

Les arbres de décision sont également capables d'effectuer des tâches de régression. Construisons un arbre de régression en utilisant la classe `DecisionTreeRegressor` de Scikit-Learn, en l'entraînant sur un ensemble de données quadratiques bruité avec `max_depth=2`: 

### Exemple

In [ ]:
# data creation
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

tree_reg = sklearn.tree.DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

In [ ]:
dot_data =sklearn.tree.export_graphviz(
        tree_reg,
        out_file=None,
        rounded=True,
        filled=True
    )

graph = graphviz.Source(dot_data) 
graph

Cet arbre ressemble beaucoup à l'arbre de classification que vous avez construit plus tôt. La principale différence est qu'au lieu de prédire une classe dans chaque nœud, il prédit une valeur. 

Par exemple, supposons que vous vouliez faire une prédiction pour une nouvelle instance avec `x[0]` = 0.6. Vous traversez l'arbre en commençant par la racine, et vous atteignez finalement le noeud leaf qui prédit `valeur=0.1106`. Cette prédiction est simplement la valeur cible moyenne des 110 instances de formation associées à ce noeud leaf. Cette prédiction se traduit par une erreur quadratique moyenne (MSE) égale à 0,0151 sur ces 110 instances. 

### Plot

In [ ]:
tree_reg1 = sklearn.tree.DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = sklearn.tree.DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel(r"$x_0$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

plt.figure(figsize=(15, 4))
plt.subplot(121)
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.subplot(122)
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14);

### L'algorithme CART 

L'algorithme CART fonctionne essentiellement de la même manière que précédemment, sauf  qu'il essaie maintenant de diviser l'ensemble train en minimiseant la MSE intra-nœud. Plus précisément l'algo minimise la quantité suivante:
$$
\frac {m_{\text{left}} } {m} \,  \text{MSE}_{\text{left}} +  \frac {m_{\text{right}} } {m} \,  \text{MSE}_{\text{right}}
$$
où
$$
\text{MSE}_{\text{left}} = \sum_{i \in \text{left}} \big( \bar y_{\text{left}} - y_i \big)^2
$$
$$
\text{MSE}_{\text{right}} = \sum_{i \in \text{right}} \big( \bar y_{\text{right}} - y_i \big)^2
$$
où $\bar y_{\text{left/right}}$ est la moyenne un le noeud de droite ou de gauche. 


### Régularisation

Tout comme pour les tâches de classification, les arbres de décision ont tendance à être surdimensionnés lorsqu'il s'agit de tâches de régression. Sans aucune régularisation (c'est-à-dire en utilisant les hyperparamètres par défaut), il sur-apprend. Le simple fait de paramétrer `min_samples_leaf=10` donne un modèle beaucoup plus raisonnable. 

In [ ]:
tree_reg1 = sklearn.tree.DecisionTreeRegressor(random_state=42)
tree_reg2 = sklearn.tree.DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

x1 = np.linspace(0, 1, 500).reshape(-1, 1)
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel(r"$x_0$", fontsize=18)
plt.ylabel("$y$", fontsize=18, rotation=0)
plt.legend(loc="upper center", fontsize=18)
plt.title("No restrictions", fontsize=14)

plt.subplot(122)
plt.plot(X, y, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.title("min_samples_leaf={}".format(tree_reg2.min_samples_leaf), fontsize=14);

## Instabilité $\hookleftarrow$



### Rotation des données

Nous espérons que vous êtes maintenant convaincu que les arbres de décision ont beaucoup d'atouts: ils sont simples à comprendre et à interpréter, faciles à utiliser, polyvalents et puissants. Cependant, ils présentent quelques limites. Tout d'abord, comme vous l'avez peut-être remarqué, les arbres décisionnels aiment les frontières de décision orthogonales (toutes les divisions sont perpendiculaires à un axe), ce qui les rend sensibles à la rotation des ensembles d'entraînement. Par exemple, voici un simple ensemble de données séparables linéairement : à gauche, un arbre de décision peut le diviser facilement, tandis qu'à droite, après une rotation de 45° de l'ensemble de données, la frontière de décision semble inutilement alambiquée. 

In [ ]:
np.random.seed(6)
Xs = np.random.rand(100, 2) - 0.5
ys = (Xs[:, 0] > 0).astype(np.float32) * 2

angle = np.pi / 4
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xsr = Xs.dot(rotation_matrix)

tree_clf_s = sklearn.tree.DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(Xs, ys)
tree_clf_sr = sklearn.tree.DecisionTreeClassifier(random_state=42)
tree_clf_sr.fit(Xsr, ys)

fig,(ax0,ax1)=plt.subplots(1,2,figsize=(12,6))

plot_decision_boundary(ax0,tree_clf_s, Xs, ys, axes=[-0.7, 0.7, -0.7, 0.7], iris=False)
plot_decision_boundary(ax1,tree_clf_sr, Xsr, ys, axes=[-0.7, 0.7, -0.7, 0.7], iris=False);

Bien que les deux arbres de décision s'adaptent parfaitement à l'ensemble de formation, il est très probable que le modèle de droite ne se généralisera pas bien. Une façon de limiter ce problème est d'utiliser l'ACP, qui permet souvent de mieux orienter les données.

### petites variations

Plus généralement, le principal problème des arbres décisionnels est qu'ils sont très sensibles aux petites variations des données relatives au jeu `train`. Par exemple, retirons une iris du jeu train.  




In [ ]:
fig,ax=plt.subplots(figsize=(16,6))
plot_decision_boundary(ax,tree_clf, X_iris, y_iris)

In [ ]:
iris = sklearn.datasets.load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target
X[(X[:, 1]==X[:, 1][y==1].max()) & (y==1)] # widest Iris-Versicolor flower
not_widest_versicolor = (X[:, 1]!=1.8) | (y==2)
X_tweaked = X[not_widest_versicolor]
y_tweaked = y[not_widest_versicolor]

tree_clf_tweaked = sklearn.tree.DecisionTreeClassifier(max_depth=2, random_state=40)
tree_clf_tweaked.fit(X_tweaked, y_tweaked)

In [ ]:
fig,ax=plt.subplots(figsize=(16,6))
plot_decision_boundary(ax,tree_clf_tweaked, X_tweaked, y_tweaked, legend=False)

En fait, puisque l'algorithme d'entraînement utilisé par Scikit-Learn est stochastique, vous pouvez obtenir des modèles très différents même sur les mêmes données d'entraînement (à moins que vous ne définissiez l'hyperparamètre random_state).  Random Forests peut limiter cette instabilité en faisant la moyenne des prévisions sur de nombreux arbres. 